In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import gc
from captum.attr import *
import quantus
import gc
import torchvision.transforms as transforms
import os
import torch.optim as optim
import torchvision.models as models
import sklearn
from sklearn.metrics import roc_auc_score

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
class Network(nn.Module):
    ''' A basic neural network model '''
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()         #python2 : super(MLP, self).__init__()
        #defining the network's operations
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], output_size)

    def forward(self, x, softmax=False): 
        x = x.reshape(-1,47)
        a = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x.float())))))
        if softmax:
            y_pred = F.softmax(a, dim=1)
        else:
            y_pred = a
        #print(y_pred.shape)
        return y_pred

In [5]:
# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on : {}".format(device))

Training on : cuda


In [6]:
input_size = 47
hidden_size=[64,64]
output_size=2
model = Network(input_size, hidden_size, output_size)
model

Network(
  (fc1): Linear(in_features=47, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [7]:
model.to(device)

Network(
  (fc1): Linear(in_features=47, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [8]:
df = pd.read_csv('clean_examples.csv')
df.head()

,Unnamed: 0,Destination Port,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Class
0,0,80.0,0.000000,0.00000,0.0000,0.0000,0.000000,4000.000000,3.333333e+02,1.753777e+02,...,0.0,251.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,1.0
1,1,80.0,54.666668,133.90544,2898.7500,4095.5662,119.999920,0.100646,1.100000e+07,3.310000e+07,...,11595.0,251.0,235.0,1.0,32.0,4.0,0.0,4.0,4.0,1.0
2,2,80.0,115.666664,173.50000,1656.4286,1547.7377,125.453190,0.158852,6.714855e+06,2.580000e+07,...,11595.0,274.0,235.0,3.0,32.0,1003.0,0.0,1003.0,1003.0,1.0
3,3,80.0,6.000000,0.00000,0.0000,0.0000,8.818477,1.469746,8.504870e+05,1.697628e+06,...,0.0,256.0,-1.0,4.0,20.0,0.0,0.0,0.0,0.0,1.0
4,4,80.0,100.000000,173.20508,1932.5000,1977.8129,83350.850000,63.064957,1.783875e+04,4.638334e+04,...,11595.0,29200.0,235.0,1.0,32.0,0.0,0.0,0.0,0.0,1.0


In [10]:
def compute_TPR(adv, th): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in adv: 
        if value>th:
            TP += 1
        else: 
            FN += 1
    
    
    return (TP/(TP+FN))*100

In [11]:
def compute_FPR(ben, th): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in ben: 
        if value<th:
            TN += 1
        else: 
            FP += 1
    
    
    return (FP/(FP+TN))*100


In [12]:
def compute_iqr(attr):
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        score_75 = np.percentile(a, 75)
        #print(score_75)
        score_25 = np.percentile(a, 25)
        #print(score_25)
        score_qt = score_75 - score_25
       # print(score_qt)
        scores.append(score_qt)
    return scores

In [13]:
def train_fs(model, x_batch, y_batch, train_fpr):
    distances = get_attr(model, x_batch, y_batch)
    selected_distance_idx = int(np.ceil(len(x_batch) * (1-train_fpr)))
    threshold = sorted(distances)[selected_distance_idx-1]
    threshold = threshold
    return threshold

def get_attr(model, x_batch, y_batch):
    a_batch_intgrad = quantus.explain(
            model=model, inputs=x_batch, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
    iqr = compute_iqr(a_batch_intgrad)
    return iqr

In [14]:
#evaluate on benign samples and save metrics
def compute_benign_metrics(test_loader, model):
    fs = []
    for image, label in test_loader: 
        image, label = image.to(device), label.to(device)
        d = get_attr(model, image, label)    
        fs.extend(d)
    return fs


#evaluate on adv samples and save metrics
def compute_adv_metrics(test_loader, model):
    fs = []
    for image, label in test_loader: 
        image, label = image.to(device), label.to(device)
        d = get_attr(model, image, label)    
        fs.extend(d)
    return fs

In [15]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.data = self.data.drop(self.data.columns[0], axis=1)  # Remove unnecessary index column
        self.features = self.data.iloc[:, :-1].values  # Features (all columns except the last)
        self.labels = self.data.iloc[:, -1].values     # Labels (last column)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# Load CSV file and split into train and test sets
dataset = CustomDataset('clean_examples.csv')
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [16]:
FPR = [0.01,0.05,0.1]
final_th = []
for fpr in FPR:
    t=[]
    for step, (x_batch, y_batch) in enumerate(train_loader):
        
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        threshold = train_fs(model, x_batch, y_batch, fpr)
        t.append(threshold.item())
        if step==50:
            break
    th = sum(t)/len(t)
    print("Threshold for {} FPR is {}.".format(fpr, th))
    final_th.append(th)

Threshold for 0.01 FPR is 0.02216381465504859.
Threshold for 0.05 FPR is 0.0194647861872891.
Threshold for 0.1 FPR is 0.018963445242786526.


In [29]:
threshold = [0.022,0.021,0.019,0.017,0.016,0.015,0.013,0.010,0.008,0.006,0.003,0.001,0.0001]

In [30]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.99, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [31]:
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_FGSM2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.99, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [32]:
np.mean(sc), np.std(sc)

(0.5509811745841291, 0.060412886508727115)

In [34]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_PGD1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [35]:
dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_PGD2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [36]:
np.mean(sc), np.std(sc)

(0.5135758612338803, 0.055162408427680554)

In [37]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [38]:

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_BIM2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [39]:
np.mean(sc), np.std(sc)

(0.4445235969001635, 0.0)

In [40]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW1.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [41]:

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_CW2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [42]:
np.mean(sc), np.std(sc)

(0.4297269479929769, 0.0)

In [43]:
sc = []

dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_autoPGD.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [44]:


dist_ben = compute_benign_metrics(test_loader, model)

advdataset = CustomDataset('adversarial_examples_autoPGD2.csv')
advtrain_data, advtest_data = train_test_split(advdataset, test_size=0.2, random_state=42)

# Create data loaders for training and testing
batch_size = 64
train_loader_adv = DataLoader(advtrain_data, batch_size=batch_size, shuffle=True)
test_loader_adv = DataLoader(advtest_data, batch_size=batch_size, shuffle=False)

dist_adv = compute_adv_metrics(test_loader_adv, model)

fpr_results = []
for th in threshold:
    FPR = compute_FPR(dist_ben, th)
    fpr_results.append(FPR/100)
tpr_results = []
for th in threshold:
    TPR = compute_TPR(dist_adv, th)
    tpr_results.append(TPR/100)
    
score = sklearn.metrics.auc(fpr_results, tpr_results)
sc.append(score)

In [45]:
np.mean(score), np.std(score)

(0.5724471756372222, 0.0)